In this series of notebooks, I will attempt to do some introductory exploration of various roller derby statistics. We will use the publicly available stats on the FlatTrackStats website. First, I will build a table scraper tool using the BeautifulSoup4 package to parse the stats tables on the website. If not already installed, you will need pandas and BeautifulSoup4 in order to run this notebook. 

In [9]:
    import requests
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from itertools import product
    import urllib.request
    import urllib.parse
    import statistics
    from statistics import mode
    import collections
    import re
    
    #First, define a class to parse HTML tables for bouts and players
    
    class HTMLTableParser:
        def parse_url(self, url):
            response = requests.get(url, timeout=None)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.read_table(table)) for table in soup.find_all('table')]      
        
        def read_table(self, table_tag):
            rowspans = []  # track pending rowspans
            rows = table_tag.find_all('tr')

            # first scan, see how many columns we need
            colcount = 0
            column_names=[]
            for r, row in enumerate(rows):
                cells = row.find_all(['td', 'th'], recursive=False)
                colcount = max(
                    colcount,
                    sum(int(c.get('colspan', 1)) or 1 for c in cells[:-1]) + len(cells[-1:]) + len(rowspans))
                # update rowspan bookkeeping; 0 is a span to the bottom. 
                rowspans += [int(c.get('rowspan', 1)) or len(rows) - r for c in cells]
                rowspans = [s - 1 for s in rowspans if s > 1]

            # it doesn't matter if there are still rowspan numbers 'active'; no extra
            # rows to show in the table means the larger than 1 rowspan numbers in the
            # last table row are ignored.    
            # build an empty matrix for all possible cells
            table = [[None] * colcount for row in rows]


            # fill matrix from row data
            rowspans = {}  # track pending rowspans, column number mapping to count
            for row, row_elem in enumerate(rows):
                span_offset = 0  # how many columns are skipped due to row and colspans 
                for col, cell in enumerate(row_elem.find_all(['td', 'th'], recursive=False)):
                    # adjust for preceding row and colspans
                    col += span_offset
                    while rowspans.get(col, 0):
                        span_offset += 1
                        col += 1

                    # fill table data
                    rowspan = rowspans[col] = int(cell.get('rowspan', 1)) or len(rows) - row
                    colspan = int(cell.get('colspan', 1)) or colcount - col
                    # next column is offset by the colspan
                    span_offset += colspan - 1
                    value = cell.get_text()
                    points = len(cell.find_all("div", {"class": " point"}))
                    pens = cell.find_all("div", {"class": "penalty major"})  
                    if (value == '' or value == '&nbsp') and points != '':
                        value = str(points)
                    if len(pens) > 0:
                        value = ''
                        for pen in pens:
                            value += pen.find_all(text=True, recursive=False)[0] + " "
                    if len(cell.find_all("div", {"class": "lead"}))== 1:
                        if value == "0": value = ""
                        value += "Lead "
                    if len(cell.find_all("div", {"class": "leadloss"}))== 1:
                        if value == "0": value = ""
                        value += "LeadLoss "
                    if len(cell.find_all("div", {"class": "lost"}))== 1:
                        if value == "0": value = ""
                        value += "LeadLoss "                   
                    if len(cell.find_all("div", {"class": "call"}))== 1:
                        if value == "0": value = ""
                        value += "call "
                    if len(cell.find_all("div", {"class": "nopass"}))== 1:
                        if value == "0": value = ""
                        value += "np "    
                    if len(cell.find_all("div", {"class": "starpass"}))== 1:
                        if value == "0": value = ""
                        value += "sp "    
                    for drow, dcol in product(range(rowspan), range(colspan)):
                        try:
                            table[row + drow][col + dcol] = value
                            rowspans[col + dcol] = rowspan
                        except IndexError:
                            # rowspan or colspan outside the confines of the table
                            pass

                # update rowspan bookkeeping
                rowspans = {c: s - 1 for c, s in rowspans.items() if s > 1}
            npt = np.array(table)
            #df = pd.DataFrame(np.array(table), column_names) 
            return table

        def parse_url_todf(self, url):
            tables = self.parse_url(url)
            dfs = []
            if "combos" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)
            if "teams" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)
            if "jams" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)        
            else:
                headers = tables[0]
                headersnew = []
                for header in headers:
                    headernew = [i for i in header if i] 
                    headersnew.append(headernew)
                for i in range(len(tables)-1):
                    df = pd.DataFrame(np.array(tables[i+1]), columns=headersnew[i])    
                    dfs.append(df)
                
            return dfs    

Let's make a function to get all of a team's bouts from the database:

In [10]:
def GetAllBouts(teamID):
    npages = 0
    links=[]
    
    base_url = "http://www.flattrackstats.com/teams/"+teamID+"/bouts"
    text = urllib.request.urlopen(base_url, timeout=None)
    base_soup = BeautifulSoup(text)
    
    for listitem in base_soup.findAll('li', class_="pager-last last"):
        npages = int(listitem.findAll('a')[0]['href'][-1])
    
    
    for page in range(0,npages):
        url = "http://www.flattrackstats.com/teams/"+teamID+"/bouts?page="+str(page)
        text = urllib.request.urlopen(url, timeout=None)
        soup = BeautifulSoup(text)
        for link in soup.findAll('a', class_="boutlink has-stats"):
            linkname = link['href']
            linksize = len(linkname)
            if "node" not in link['href']:
                linkstub = linkname[:linksize-8]
            else: linkstub = "/bouts"+linkname[5:]+"/"
            links.append(linkstub)
                
    return links

In [11]:
#GetAllBouts(str(3637))

Now let's make some functions to read the data from the bouts page (one such page is here: http://flattrackstats.com/bouts/55689/jams).

The first will be a function to read the table of points scored in each jam, logged per lap. This is similar in format to what is recorded on the track. Let's make sure to drop the opposing team's scores from this.

In [12]:
def GetAllScores(teamID, teamName):
    scoreframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://www.flattrackstats.com/"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage, timeout = None)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True
        
        scores = hp.parse_url_todf(jampage)[1].iloc[:,1:-1]
        scores.loc[np.c_[scores.index[1::2].tolist(),scores.index[0:-1:2].tolist()].reshape(-1)]
        
        #del scores[0]
        #print(scores)
        scoreframes.append(scores)
    allScores = pd.concat(scoreframes)
    #drop opposing team
    allScores = allScores.iloc[::2]
    allScores = allScores.reset_index()
    del allScores['index']
    #print(allScores)
    return(allScores)

Next, let's make a function to get the table recording who played in each jam, and what the cumulative score, jam score, and score differential (my_team - opposing_team) was for that jam. Again, let's drop the opposing team's lineups. This is the function that will give us most of our data!

In [13]:
def GetAllLineups(teamID, teamName):
    lineupframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://www.flattrackstats.com/"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage, timeout = None)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True        
        lineups = hp.parse_url_todf(jampage)[2].iloc[:,1:-1]
        
        #put team of interest first (data is naturally in format (home, away) )
        if isAway:
            lineups = lineups.loc[np.c_[lineups.index[1::2].tolist(),lineups.index[0:-1:2].tolist()].reshape(-1)]
        lineupframes.append(lineups) 
        
    #clean data    
    allLineups = pd.concat(lineupframes)
    allLineups.columns = ['Jammer','Jstats','B1','B2','B3','B4','NULL','jamscore','runscore']
    del allLineups['NULL']
    
    allLineups = allLineups[~(allLineups.jamscore == 'Period 2')]
    allLineups = allLineups[~(allLineups.jamscore.isnull())]
    
    #calc score differentials, for determining rankings
    allLineups['jamscore'] = pd.to_numeric(allLineups['jamscore'])
    allLineups['runscore'] = pd.to_numeric(allLineups['runscore'])
    
    allLineups['ScoreDiff'] = allLineups['jamscore'].diff(periods=-1)
    
    #drop opposing team
    allLineups = allLineups.iloc[::2]
    allLineups = allLineups.reset_index()
    del allLineups['index']
    
    return(allLineups)

Next, let's make a function to get the penalties per jam. This will need to be cross-referenced with the lineups table in order to determine who got what penalties.

In [14]:
def GetAllPenalties(teamID, teamName):
    penframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://www.flattrackstats.com/"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage, timeout=None)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True
        
        pens = hp.parse_url_todf(jampage)[3].iloc[:,1:-1]
        
        if isAway:
            pens.loc[np.c_[pens.index[1::2].tolist(),pens.index[0:-1:2].tolist()].reshape(-1)]

        #del pens[0]
        penframes.append(scores)
    allPens = pd.concat(penframes)
    #print(allPens)
    return(allPens)

In lieu of rankings like ELO, let's just use an average of the score differential for each blocker (something like ELO would be better, but cumbersome to caclulate right now). Let's get an expanding average of the blocker's point differential (that is, after a given jam, the point differential for each jam the blocker has played in, averaged over all the times they have played).

After calculating the expanding averages, let's also drop all jams for which one or more blockers involved hasn't played in at least 12 jams (that is, roughly one half a bout) from the dataset. This gives each blocker's average score differential a chance to stabilize. This block also has some other functions that might be useful for calculating the useful metrics for each jam.

In [15]:
def ExpandingAverages(teamID, teamName):
    lineups = GetAllLineups(teamID,teamName)
    allBlockers = (lineups['B1'].append(lineups['B2']).append(lineups['B3']).append(lineups['B4'])).unique()
    #print(allBlockers, len(allBlockers))
    blockerCols = lineups[['B1','B2','B3','B4']]
    #print(blockerCols)
    blockerlist = pd.concat([pd.get_dummies(blockerCols[col]) for col in blockerCols.columns], axis = 1)
    indicators = blockerlist.groupby(by=blockerlist.columns, axis=1).sum()
    scorediffs = indicators.mul(lineups['ScoreDiff'], axis=0)
    #don't include unplayed jams
    scorediffs.replace(0, np.nan, inplace=True)
    runningmean = scorediffs.expanding(1).mean()
    runningmean.replace(np.nan, 0, inplace=True)
    pd.set_option('display.max_columns', 58)  # or 1000
    pd.set_option('display.max_rows', 20)  # or 1000
    pd.set_option('display.max_colwidth', 15)  # or 199
    #test2 = pd.concat([pd.get_dummies(blockerCols[col]) for col in blockerCols.columns], axis=1)
    #test = lineups.stack().str.get_dummies(columns =['B1','B2','B3','B4']).sum(level=0)
    return(runningmean)

In [16]:
# This function just gets a player's running score differential (i.e., if a player scores +1 and then -1, after those two jams,
# it returns zero)

def RunningDiffs(teamID, teamName):
    lineups = GetAllLineups(teamID,teamName)
    allBlockers = (lineups['B1'].append(lineups['B2']).append(lineups['B3']).append(lineups['B4'])).unique()
    #print(allBlockers, len(allBlockers))
    blockerCols = lineups[['B1','B2','B3','B4']]
    #print(blockerCols)
    blockerlist = pd.concat([pd.get_dummies(blockerCols[col]) for col in blockerCols.columns], axis = 1)
    indicators = blockerlist.groupby(by=blockerlist.columns, axis=1).sum()
    scorediffs = indicators.mul(lineups['ScoreDiff'], axis=0)
    scorediffs.replace(0, np.nan, inplace=True)
    pd.set_option('display.max_columns', 58)  # or 1000
    pd.set_option('display.max_rows', 20)  # or 1000
    pd.set_option('display.max_colwidth', 15)  # or 199
    #test2 = pd.concat([pd.get_dummies(blockerCols[col]) for col in blockerCols.columns], axis=1)
    #test = lineups.stack().str.get_dummies(columns =['B1','B2','B3','B4']).sum(level=0)
    return(scorediffs)

In [17]:
#This function returns a running list of how much a given jam changes a player's average

def AvgChangeFromJam(teamID, teamName):
    avgs = ExpandingAverages(teamID, teamName)
    avgswzero = pd.DataFrame([[0.0] * len(avgs.columns)], columns=avgs.columns)
    avgs = avgswzero.append(avgs, ignore_index=True)
    deltas = avgs.diff()
    deltas = deltas.drop([0])
    return deltas

In [18]:
#This function returns a list of how many total jams a player has played at a particular time

def JamsPlayed(teamID, teamName):
    lineups = GetAllLineups(teamID,teamName)
    allBlockers = (lineups['B1'].append(lineups['B2']).append(lineups['B3']).append(lineups['B4'])).unique()
    #print(allBlockers, len(allBlockers))
    blockerCols = lineups[['B1','B2','B3','B4']]
    #print(blockerCols)
    blockerlist = pd.concat([pd.get_dummies(blockerCols[col]) for col in blockerCols.columns], axis = 1)
    indicators = blockerlist.groupby(by=blockerlist.columns, axis=1).sum()
    indicators.replace(0, np.nan, inplace=True)
    jamtot = indicators.expanding(1).sum()
    return(jamtot)

In [19]:
#This function returns a list of all jams for which any player involved has not played N jams,
#as well as a list of blockers who play a total of fewer than N jams.

def GetBadJamsAndBlockers(teamID, teamName, N=12):
    lineups = GetAllLineups(teamID,teamName)
    jp = JamsPlayed(teamID,teamName)
    allBlockers = (lineups['B1'].append(lineups['B2']).append(lineups['B3']).append(lineups['B4'])).unique()
    #print(allBlockers, len(allBlockers))
    blockerCols = lineups[['B1','B2','B3','B4']]
    #print(blockerCols)
    badjamlist = []
    
    for jamnum in range(len(jp)):
        for col in blockerCols.columns:
            blockername = blockerCols.loc[jamnum,col]
            jamcount = jp.loc[jamnum,blockername]
            if (jamcount < N): 
                badjamlist.append(jamnum)
    
    mask = jp.iloc[-1] < N
    badjp = jp.loc[:, mask]
    badblockerlist=list(badjp.columns)
    
    badjamarray = np.array(badjamlist)
    badjamarray=np.unique(badjamarray)
    
    return(badjamarray.tolist(),badblockerlist)

Now that we've got all our functions, let's do some preprocessing. First, we want to make sure that the ExpandingAverage is a relatively stable metric of a player's skill, and determine approximately the number of jams played that is needed for the metric to be valid. Let's look at Killamazoo:

In [20]:
#Remove bad jams and blockers

def ExpandingAveragesCleaned(teamID, teamName,N=12):
    avgs = ExpandingAverages(teamID, teamName)
    badjams,badblockers = GetBadJamsAndBlockers(teamID, teamName,N)
    avgs= avgs.drop(labels=badjams, axis=0)
    avgs= avgs.drop(labels=badblockers, axis=1)
    
    return(avgs)

In [6]:
#exa12= ExpandingAveragesCleaned(str(3637),'Killamazoo',12)
#exa12

In [7]:
#exa20= ExpandingAveragesCleaned(str(3637),'Killamazoo',20)
#exa20

In [8]:
#exa30= ExpandingAveragesCleaned(str(3637),'Killamazoo',30)
#exa30

NameError: name 'ExpandingAverages' is not defined

Ok, let's use 12 as our minimum number of jams and plot the expanding averages for Javelin, one of Kalamazoo's longest-playing blockers.

In [ ]:
exa20["Javelin"].plot()

Okay great! This seems to be pretty good- we can see her steadily improving over time, starting out circa -5 and getting to a differential of zero. This is a solid metric. If we wanted to, we could compare this to ELO and other ranking systems to determine accuracy, calculating how predictive the various metrics are- but this one is easy to calculate and seems decent enough for now.

Finally, let's make a function to get all the teams and their names.

In [20]:
def getNameFromID(teamID):
    npages = 0
    links=[]
    scoreframes = []
    allteams=[]
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for i,bout in enumerate(bouts):
        jampage = "http://www.flattrackstats.com/"+bout+"jams" 
        isAway = False 
        response = requests.get(jampage, timeout=None)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            awayteam = fulltitle[fulltitle.find(': ')+1 : fulltitle.find(' (')]
            hometeam = fulltitle[fulltitle.find('@ ')+1 : fulltitle.find(' (', fulltitle.find(' (') + 1)]
            allteams.append(hometeam)
            allteams.append(awayteam)
        if i > 10: break    
    if (allteams==[]): teamname = "NO_STATS"        
    else: teamname=mode(allteams)    
    
    #print(teamname)            
    return teamname        

In [21]:
def getAllTeamsAndNames():
    npages = 0
    IDs_All=[]
    Names_All=[]
    IDs=[]
    Names=[]
    
    base_url = "http://www.flattrackstats.com/teams/results/wftda-usa"
    text = urllib.request.urlopen(base_url, timeout=None)
    base_soup = BeautifulSoup(text)
    
    for listitem in base_soup.findAll('li', class_="pager-last last"):
        npages = int(listitem.findAll('a')[0]['href'][-1])
    
    
    for page in range(0,npages):
        url = base_url+"?page="+str(page)
        text = urllib.request.urlopen(url, timeout=None)
        soup = BeautifulSoup(text)
        for link in soup.findAll('a', class_=''):
            linkname = link['href']
            #print(linkname)
            linksize = len(linkname)
            if "node" in link['href']:
                linkstub = linkname[6:linksize]
                #print(linkstub)
                IDs_All.append(linkstub)
            elif "overview" in link['href']:
                linkstub = linkname[7:linksize-9]
                #print(linkstub)
                IDs_All.append(linkstub)
            
    for ID in IDs_All:
        #print(ID)
        name = getNameFromID(ID)
        #print(name)
        Names_All.append(name)
   
    for ID, name in zip(IDs_All,Names_All):
        if name == "NO_STATS": continue
        IDs.append(ID)    
        Names.append(name)

    return IDs,Names
